# Eksperymenty na CORA: Prosty Autoenkoder, Porównanie propocji maskowania, Porównanie strategii maskowania, Porównanie z innymi metodami maskowania

# Przygotowanie

In [10]:
!python --version

Python 3.10.12


In [11]:
!pip install networkx

In [12]:
!pip install torch

In [13]:
!pip install torch-geometric

In [14]:
!pip install umap-learn

In [15]:
!pip install scikit-learn

In [16]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.33.0
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.14
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.3
astropy-iers-data                0.2024.8.27.10.28.29
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread             

# Graf losowy syntetyczny - MaskGAE 2GCNConv

## Przewidywanie połaczeń

In [ ]:
import torch
import networkx as nx
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from tqdm import tqdm

CUSTOM_EPOCHS = 60

# Sprawdzenie dostępności CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Wczytanie zestawu danych Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)

# Podział na zbiór treningowy i testowy
def split_edges(data, test_ratio=0.2):
    edge_index = data.edge_index.to(device)
    num_edges = edge_index.size(1)

    # Wygenerowanie losowego permutacji indeksów
    perm = torch.randperm(num_edges).to(device)
    test_size = int(num_edges * test_ratio)

    # Podział indeksów na zbiór testowy i treningowy
    test_edge_index = edge_index[:, perm[:test_size]]
    train_edge_index = edge_index[:, perm[test_size:]]

    # Utworzenie dwóch obiektów Data: jeden dla treningu, drugi dla testowania
    train_data = Data(x=data.x, edge_index=train_edge_index).to(device)
    test_data = Data(x=data.x, edge_index=test_edge_index).to(device)

    return train_data, test_data

train_data, test_data = split_edges(data, test_ratio=0.2)

# Implementacja Autoenkodera Grafowego MaskGAE
class MaskGAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MaskGAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)
        self.decoder = GCNConv(out_channels, in_channels)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return self.decode(z, edge_index)

# Maskowanie krawędzi
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long).to(device)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)

    data.edge_index = masked_edge_index
    return data

# Nowa strategia: maskowanie sąsiedztwa (neighborhood)
def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)
    data.edge_index = masked_edge_index
    return data

# Nowa strategia: maskowanie atrybutów (attributes)
def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long).to(device)

    data.x[mask_indices] = 0  # Zerowanie atrybutów wybranych węzłów
    return data

# Zaktualizowana funkcja trenowania z nowymi strategiami maskowania
def train_and_evaluate(in_channels, out_channels, train_data, mask_ratios, epochs=100, mask_strategy='mask_edges'):
    results = defaultdict(dict)
    models = {}
    criterion = torch.nn.MSELoss()

    for ratio in mask_ratios:
        print(f'Training with mask ratio: {ratio} using {mask_strategy} strategy')

        model = MaskGAE(in_channels=in_channels, out_channels=out_channels).to(device)
        optimizer = Adam(model.parameters(), lr=0.01)

        losses = []

        for epoch in tqdm(range(epochs), desc=f'Training for {mask_strategy}, mask ratio {ratio}'):
            model.train()
            optimizer.zero_grad()

            # Wybór odpowiedniej strategii maskowania
            if mask_strategy == 'mask_edges':
                masked_train_data = mask_edges(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_neighborhood':
                masked_train_data = mask_neighborhood(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_attributes':
                masked_train_data = mask_attributes(train_data.clone(), ratio) if ratio > 0 else train_data

            out = model(masked_train_data.x, masked_train_data.edge_index)
            loss = criterion(out, train_data.x)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())
            tqdm.write(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

        results[ratio] = losses
        models[(mask_strategy, ratio)] = model

    return results, models

# Funkcja do oceny modelu na zbiorze testowym
def evaluate_on_test(model, test_data):
    model.eval()
    with torch.no_grad():
        z = model.encode(test_data.x, test_data.edge_index)
        out = model.decode(z, test_data.edge_index)
        loss = calculate_prediction_error(out, test_data)
        print(f'Test Loss: {loss}')
        return loss

def calculate_prediction_error(pred, data):
    return torch.nn.functional.mse_loss(pred, data.x).item()

# Zaktualizowane funkcje wizualizacyjne
def plot_results(results):
    plt.figure(figsize=(15, 10))
    mask_strategies = results.keys()

    for i, mask_strategy in enumerate(mask_strategies):
        plt.subplot(2, 2, i + 1)
        strategy_results = results[mask_strategy]  # Get the dictionary for the strategy directly
        for ratio, losses in strategy_results.items():
            if isinstance(losses, list):  # Ensure losses is a list
                if ratio == 0.0:
                    plt.plot(losses, label=f'Without masking')
                else:
                    plt.plot(losses, label=f'Mask ratio {ratio}')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title(f'MaskGAE 2GCN: {mask_strategy.capitalize()} Strategy: Effect of Masking Ratio on MSE Loss')
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()

def final_loss_comparison(results):
    mask_strategies = results.keys()

    for mask_strategy in mask_strategies:
        final_losses = {ratio: losses[-1] for ratio, losses in results[mask_strategy].items()}
        ratios = list(final_losses.keys())
        losses = list(final_losses.values())

        plt.figure(figsize=(10, 6))
        plt.bar(ratios, losses, color='skyblue')
        plt.xlabel('Mask Ratio')
        plt.ylabel('Final MSE Loss')
        plt.title(f'MaskGAE 2GCN: {mask_strategy.capitalize()} Strategy: Final MSE Loss for Different Masking Ratios')
        plt.grid(True)
        plt.show()

# Funkcja predykcji na nieoznaczonych danych
def predict(model, data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_index)
        return out, z

# Nowa funkcja do obliczania miar
def calculate_metrics(pred, true):
    # Move tensors to CPU and convert to numpy
    true = true.cpu().numpy().flatten()
    pred = pred.cpu().numpy().flatten()

    # Compute ROC curve and optimal threshold
    fpr, tpr, thresholds = roc_curve(true, pred)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # Use optimal threshold to generate binary predictions
    pred_binary = (pred > optimal_threshold).astype(float)
    true_binary = (true > 0.5).astype(float)

    # print(f'true_binary = {true_binary}\n')
    # print(f'pred = {pred}\n')

    # Calculate various metrics
    mse = F.mse_loss(torch.tensor(pred), torch.tensor(true)).item()
    f1 = f1_score(true_binary, pred_binary, average='macro')
    accuracy = accuracy_score(true_binary, pred_binary)
    precision = precision_score(true_binary, pred_binary, average='macro')
    recall = recall_score(true_binary, pred_binary, average='macro')
    roc_auc = roc_auc_score(true_binary, pred, average='macro')
    avg_precision = average_precision_score(true_binary, pred, average='macro')

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

# Trening i ocena modelu
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
strategies = ['mask_edges', 'mask_neighborhood', 'mask_attributes']

# Trening i ocena modelu z nowymi strategiami
all_results = {}
all_models = {}
for strategy in strategies:
    results, models = train_and_evaluate(
        in_channels=train_data.num_features,
        out_channels=2,
        train_data=train_data,
        mask_ratios=mask_ratios,
        epochs=CUSTOM_EPOCHS,
        mask_strategy=strategy
    )
    all_results[strategy] = results
    all_models[strategy] = models

# Wyświetlanie wyników
plot_results(all_results)
final_loss_comparison(all_results)

# Ewaluacja na zbiorze testowym
for strategy in strategies:
    for ratio in mask_ratios:
        model = all_models[strategy][(strategy, ratio)]
        test_loss = evaluate_on_test(model, test_data)
        print(f"Strategy: {strategy}, Mask Ratio: {ratio}, Test Loss: {test_loss}")

# Predykcja na nieoznaczonych danych
unlabeled_data = test_data

predictions = {}
errors = {}
metrics = {}
for strategy in strategies:
    for ratio in mask_ratios:
        model = all_models[strategy][(strategy, ratio)]
        pred, _ = predict(model, unlabeled_data)
        error = calculate_prediction_error(pred, unlabeled_data)
        predictions[(strategy, ratio)] = pred
        errors[(strategy, ratio)] = error


        metric_values = calculate_metrics(pred, unlabeled_data.x)
        metrics[(strategy, ratio)] = metric_values
        print(f"Metrics for Strategy: {strategy}, Mask Ratio: {ratio}")
        print(metric_values)

# Wizualizacja błędów predykcji
plt.figure(figsize=(15, 10))
for i, strategy in enumerate(strategies):
    plt.subplot(2, 2, i + 1)
    strategy_errors = {ratio: errors[(strategy, ratio)] for ratio in mask_ratios}
    plt.bar(strategy_errors.keys(), strategy_errors.values(), color='skyblue')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Prediction Error')
    plt.title(f'MaskGAE 2GCN: {strategy.capitalize()} Strategy: Link Prediction Errors')
    plt.grid(True)

plt.tight_layout()

plt.show()

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != _repr(self.p

cpu
Training with mask ratio: 0.0 using mask_edges strategy


Training for mask_edges, mask ratio 0.0:   3%|▎         | 2/60 [00:00<00:09,  6.03it/s]

Epoch 1/60, Loss: 0.012689209543168545
Epoch 2/60, Loss: 0.012565281242132187


Training for mask_edges, mask ratio 0.0:   7%|▋         | 4/60 [00:00<00:09,  5.79it/s]

...

Epoch 3/60, Loss: 0.012433023191988468


Training for mask_neighborhood, mask ratio 0.1:   7%|▋         | 4/60 [00:02<00:38,  1.45it/s]

Epoch 4/60, Loss: 0.012354977428913116


Training for mask_neighborhood, mask ratio 0.1:   8%|▊         | 5/60 [00:03<00:37,  1.46it/s]

Epoch 5/60, Loss: 0.012310121208429337


## Klasyfikacja społeczności

In [ ]:
import torch
import networkx as nx
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from tqdm import tqdm  # Import tqdm

CUSTOM_EPOCHS = 400

# Sprawdzenie dostępności CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Załadowanie zbioru danych Cora
dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0].to(device)

# Podział na zbiór treningowy i testowy (węzły i krawędzie)
def split_edges_and_nodes(data, test_ratio=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)
    num_nodes = data.num_nodes

    # Wygenerowanie losowego permutacji indeksów
    perm_edges = torch.randperm(num_edges)
    perm_nodes = torch.randperm(num_nodes)
    test_size_edges = int(num_edges * test_ratio)
    test_size_nodes = int(num_nodes * test_ratio)

    # Podział indeksów na zbiór testowy i treningowy
    test_edge_index = edge_index[:, perm_edges[:test_size_edges]].to(device)
    train_edge_index = edge_index[:, perm_edges[test_size_edges:]].to(device)
    train_mask = torch.zeros(num_nodes, dtype=torch.bool).to(device)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool).to(device)
    train_mask[perm_nodes[test_size_nodes:]] = True
    test_mask[perm_nodes[:test_size_nodes]] = True

    train_data = Data(x=data.x, edge_index=train_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask).to(device)
    test_data = Data(x=data.x, edge_index=test_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask).to(device)

    return train_data, test_data

train_data, test_data = split_edges_and_nodes(data, test_ratio=0.2)

# Implementacja Autoenkodera Grafowego MaskGAE
class MaskGAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_classes):
        super(MaskGAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels).to(device)
        self.conv2 = GCNConv(2 * out_channels, out_channels).to(device)
        self.classifier = nn.Linear(out_channels, num_classes).to(device)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def classify(self, z):
        return self.classifier(z)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return self.classify(z)

# Maskowanie krawędzi
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()

    data.edge_index = masked_edge_index
    return data

# Nowa strategia: maskowanie sąsiedztwa (neighborhood)
def mask_neighborhood(data, mask_ratio):

    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()




    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]












        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()
    data.edge_index = masked_edge_index
    return data

# Nowa strategia: maskowanie atrybutów (attributes)
def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    data.x[mask_indices] = 0  # Zerowanie atrybutów wybranych węzłów



    return data

# Zaktualizowana funkcja trenowania z nowymi strategiami maskowania
def train_and_evaluate_classification(in_channels, out_channels, num_classes, train_data, mask_ratios, epochs=100, mask_strategy='mask_edges'):
    results = defaultdict(dict)
    models = {}
    criterion = torch.nn.CrossEntropyLoss()

    for ratio in mask_ratios:
        print(f'Training with mask ratio: {ratio} using {mask_strategy} strategy')


        model = MaskGAE(in_channels=in_channels, out_channels=out_channels, num_classes=num_classes).to(device)
        optimizer = Adam(model.parameters(), lr=0.01)

        losses = []


        for epoch in tqdm(range(epochs), desc=f'Training for {mask_strategy}, mask ratio {ratio}'):
            model.train()
            optimizer.zero_grad()

            # Wybór odpowiedniej strategii maskowania
            if mask_strategy == 'mask_edges':
                masked_train_data = mask_edges(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_neighborhood':
                masked_train_data = mask_neighborhood(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_attributes':
                masked_train_data = mask_attributes(train_data.clone(), ratio) if ratio > 0 else train_data


            out = model(masked_train_data.x, masked_train_data.edge_index)
            loss = criterion(out[train_data.train_mask], train_data.y[train_data.train_mask])
            loss.backward()
            optimizer.step()



            losses.append(loss.item())
            tqdm.write(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


        results[ratio] = losses
        models[(mask_strategy, ratio)] = model

    return results, models

# Funkcja do obliczania miar oceny
def calculate_metrics(pred, true):
    pred_binary = (pred > 0.5).float()
    true_binary = (true > 0.5).float()

    mse = F.mse_loss(pred, true).item()
    f1 = f1_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy(), average='macro')
    accuracy = accuracy_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy())
    precision = precision_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy(), average='macro')
    recall = recall_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy(), average='macro')


    roc_auc = roc_auc_score(true_binary.cpu().numpy(), pred.cpu().numpy(), average='macro')
    avg_precision = average_precision_score(true_binary.cpu().numpy(), pred.cpu().numpy(), average='macro')

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

# Funkcja do oceny modelu na zbiorze testowym
def evaluate_model_classification(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        _, pred = out.max(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = int(correct) / int(data.test_mask.sum())


        pred_prob = F.softmax(out, dim=1)[:, 1]
        metrics = calculate_metrics(pred_prob[data.test_mask], data.y[data.test_mask])

        print(f"Accuracy: {acc:.4f}, MSE: {metrics['MSE']:.4f}, F1 Score: {metrics['F1 Score']:.4f}, Precision: {metrics['Precision']:.4f}, Recall: {metrics['Recall']:.4f}, ROC AUC: {metrics['ROC AUC']:.4f}, Average Precision: {metrics['Average Precision']:.4f}")


        return metrics

# Trening i ocena modelu
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
strategies = ['mask_edges', 'mask_neighborhood', 'mask_attributes']

# Trening i ocena modelu z nowymi strategiami
all_models = {}
accuracies = defaultdict(dict)
loss_results = defaultdict(dict)

for strategy in strategies:
    results, models = train_and_evaluate_classification(
        in_channels=train_data.num_features,
        out_channels=2,
        num_classes=dataset.num_classes,
        train_data=train_data,
        mask_ratios=mask_ratios,
        epochs=CUSTOM_EPOCHS,
        mask_strategy=strategy

    )
    all_models[strategy] = models
    loss_results[strategy] = results


    for ratio in mask_ratios:
        model = models[(strategy, ratio)]
        metrics = evaluate_model_classification(model, test_data)

        print(f"Metrics for Strategy: {strategy}, Mask Ratio: {ratio}")
        print(metrics)
        accuracies[strategy][ratio] = metrics['Accuracy']

# Plot classification accuracy for each mask ratio
plt.figure(figsize=(15, 10))
for i, strategy in enumerate(strategies):
    plt.subplot(2, 2, i + 1)
    strategy_accuracies = accuracies[strategy]
    plt.bar(strategy_accuracies.keys(), strategy_accuracies.values(), color='skyblue')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Accuracy')
    plt.title(f'MaskGAE 2GCN: {strategy.capitalize()} Strategy: Classification Accuracy')
    plt.grid(True)

plt.tight_layout()
plt.show()

# Plot training loss results for each strategy and mask ratio
def plot_loss_results(loss_results):
    plt.figure(figsize=(15, 10))
    mask_strategies = loss_results.keys()

    for i, mask_strategy in enumerate(mask_strategies):
        plt.subplot(2, 2, i + 1)
        strategy_results = loss_results[mask_strategy]
        for ratio, losses in strategy_results.items():
            if isinstance(losses, list):
                plt.plot(losses, label=f'Mask ratio {ratio}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'MaskGAE 2GCN: {mask_strategy.capitalize()} Strategy: Training Loss')
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()


plot_loss_results(loss_results)

# Graf losowy syntetyczny - MaskGAE 2GCNConv 4MLP

## Przewidywanie połaczeń

In [ ]:
import torch
import networkx as nx
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from torch_geometric.datasets import Planetoid
from tqdm import tqdm

# Device configuration (CUDA or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

CUSTOM_EPOCHS = 3

# Load the Cora dataset from PyG
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)

# Split edges (train/test split)
def split_edges(data, test_ratio=0.2):
    edge_index = data.edge_index.to(device)
    num_edges = edge_index.size(1)

    # Random permutation of indices
    perm = torch.randperm(num_edges, device=device)  # Ensure perm is on the same device
    test_size = int(num_edges * test_ratio)

    # Split into test and train edges
    test_edge_index = edge_index[:, perm[:test_size]].to(device)
    train_edge_index = edge_index[:, perm[test_size:]].to(device)

    # Create two Data objects: one for training, one for testing
    train_data = Data(x=data.x, edge_index=train_edge_index).to(device)
    test_data = Data(x=data.x, edge_index=test_edge_index).to(device)

    return train_data, test_data

train_data, test_data = split_edges(data, test_ratio=0.2)

# Implementacja Autoenkodera Grafowego MaskGAE
class MaskGAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MaskGAE, self).__init__()

        # Enkoder z dwoma warstwami GCN
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

        # Dekoder struktury z dwoma warstwami MLP
        self.structure_decoder = nn.Sequential(
            nn.Linear(out_channels, 2 * out_channels),
            nn.ReLU(),
            nn.Linear(2 * out_channels, in_channels)
        )

        # Dekoder stopnia z dwoma warstwami MLP
        self.degree_decoder = nn.Sequential(
            nn.Linear(out_channels, 2 * out_channels),
            nn.ReLU(),
            nn.Linear(2 * out_channels, in_channels)
        )

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode_structure(self, z):
        # Dekodowanie struktury grafu
        return self.structure_decoder(z)

    def decode_degree(self, z):
        # Dekodowanie stopnia wierzchołków
        return self.degree_decoder(z)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        structure_reconstruction = self.decode_structure(z)
        degree_prediction = self.decode_degree(z)

        # Zwracamy rekonstrukcję struktury i predykcję stopnia jako wynik
        return structure_reconstruction, degree_prediction

# Maskowanie krawędzi
def mask_edges(data, mask_ratio):

    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)


    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()





    data.edge_index = masked_edge_index
    return data






# Maskowanie sąsiedztwa
def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)





    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()


    data.edge_index = masked_edge_index
    return data

# Maskowanie atrybutów
def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    data.x[mask_indices] = 0  # Zero out the features of selected nodes
    return data

# Trening modelu z różnymi strategiami maskowania
def train_and_evaluate(in_channels, out_channels, train_data, mask_ratios, epochs=100, mask_strategy='mask_edges'):
    results = defaultdict(dict)
    models = {}
    criterion = torch.nn.MSELoss()

    for ratio in mask_ratios:
        print(f'Training with mask ratio: {ratio} using {mask_strategy} strategy')

        # Initialize a new model for each mask ratio
        model = MaskGAE(in_channels=in_channels, out_channels=out_channels).to(device)
        optimizer = Adam(model.parameters(), lr=0.01)

        losses = []  # Collect losses for each ratio

        for epoch in tqdm(range(epochs), desc=f"Training ratio {ratio}"):
            model.train()
            optimizer.zero_grad()

            if mask_strategy == 'mask_edges':
                masked_train_data = mask_edges(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_neighborhood':
                masked_train_data = mask_neighborhood(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_attributes':
                masked_train_data = mask_attributes(train_data.clone(), ratio) if ratio > 0 else train_data

            structure_out, degree_out = model(masked_train_data.x, masked_train_data.edge_index)
            loss = criterion(structure_out, train_data.x)  # Loss is computed on the training data
            loss.backward()
            optimizer.step()

            # Print and store the loss
            losses.append(loss.item())
            tqdm.write(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

        # Store the losses and models after training
        results[ratio] = losses
        models[(mask_strategy, ratio)] = model

    return results, models

# Funkcja do oceny modelu na zbiorze testowym
def evaluate_on_test(model, test_data):
    model.eval()
    with torch.no_grad():
        z = model.encode(test_data.x, test_data.edge_index)
        structure_out = model.decode_structure(z)
        degree_out = model.decode_degree(z)
        loss = calculate_prediction_error(structure_out, test_data)
        print(f'Test Loss: {loss}')
        return loss

def calculate_prediction_error(pred, data):
    return torch.nn.functional.mse_loss(pred, data.x).item()

# Zaktualizowane funkcje wizualizacyjne
def plot_results(results):
    plt.figure(figsize=(15, 10))
    mask_strategies = results.keys()

    for i, mask_strategy in enumerate(mask_strategies):
        plt.subplot(2, 2, i + 1)
        strategy_results = results[mask_strategy]  # Get the dictionary for the strategy directly
        for ratio, losses in strategy_results.items():
            if isinstance(losses, list):  # Ensure losses is a list
                if ratio == 0.0:
                    plt.plot(losses, label=f'Without masking')
                else:
                    plt.plot(losses, label=f'Mask ratio {ratio}')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title(f'MaskGAE: {mask_strategy.capitalize()} Strategy: Effect of Masking Ratio on MSE Loss')
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()

def final_loss_comparison(results):
    mask_strategies = results.keys()

    for mask_strategy in mask_strategies:
        final_losses = {ratio: losses[-1] for ratio, losses in results[mask_strategy].items()}
        ratios = list(final_losses.keys())
        losses = list(final_losses.values())

        plt.figure(figsize=(10, 6))
        plt.bar(ratios, losses, color='skyblue')
        plt.xlabel('Mask Ratio')
        plt.ylabel('Final MSE Loss')
        plt.title(f'MaskGAE: {mask_strategy.capitalize()} Strategy: Final MSE Loss for Different Masking Ratios')
        plt.grid(True)
        plt.show()

# Funkcja predykcji na nieoznaczonych danych
def predict(model, data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
        structure_out = model.decode_structure(z)
        degree_out = model.decode_degree(z)
        return structure_out, degree_out

# Funkcja do obliczania dodatkowych metryk
def calculate_metrics(pred, data):
    mse = calculate_prediction_error(pred, data)
    pred_binary = (pred > 0.5).float()
    data_binary = (data.x > 0.5).float()

    # Przekształcenie do wektora
    pred_flat = pred_binary.flatten().cpu().numpy()
    data_flat = data_binary.flatten().cpu().numpy()

    f1 = f1_score(data_flat, pred_flat)
    accuracy = accuracy_score(data_flat, pred_flat)
    precision = precision_score(data_flat, pred_flat)
    recall = recall_score(data_flat, pred_flat)
    roc_auc = roc_auc_score(data_flat, pred.flatten().cpu().numpy())
    average_precision = average_precision_score(data_flat, pred.flatten().cpu().numpy())

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': average_precision
    }

# Trening i ocena modelu
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
strategies = ['mask_edges', 'mask_neighborhood', 'mask_attributes']

all_results = {}
all_models = {}
for strategy in strategies:
    results, models = train_and_evaluate(
        in_channels=train_data.num_features,
        out_channels=2,
        train_data=train_data,
        mask_ratios=mask_ratios,
        epochs=CUSTOM_EPOCHS,
        mask_strategy=strategy
    )
    all_results[strategy] = results
    all_models[strategy] = models

# After training
print(all_results)

plot_results(all_results)
final_loss_comparison(all_results)

# Ewaluacja na zbiorze testowym
for strategy in strategies:
    for ratio in mask_ratios:
        model = all_models[strategy][(strategy, ratio)]
        test_loss = evaluate_on_test(model, test_data)
        print(f"Strategy: {strategy}, Mask Ratio: {ratio}, Test Loss: {test_loss}")

# Predykcja na nieoznaczonych danych
unlabeled_data = test_data

predictions = {}
errors = {}
metrics_results = {}
for strategy in strategies:
    for ratio in mask_ratios:
        model = all_models[strategy][(strategy, ratio)]
        structure_pred, degree_pred = predict(model, unlabeled_data)

        # Calculate all metrics
        structure_metrics = calculate_metrics(structure_pred, unlabeled_data)
        degree_metrics = calculate_metrics(degree_pred, unlabeled_data)

        predictions[(strategy, ratio)] = (structure_pred, degree_pred)
        metrics_results[(strategy, ratio)] = {'Structure': structure_metrics, 'Degree': degree_metrics}
        errors[(strategy, ratio)] = (structure_metrics['MSE'], degree_metrics['MSE'])  # Store the errors as MSE values for plotting
        print(f"Strategy: {strategy}, Mask Ratio: {ratio}, Metrics: {metrics_results[(strategy, ratio)]}")

# Wizualizacja błędów predykcji
bar_width = 0.35
for strategy in strategies:
    strategy_errors = {ratio: errors[(strategy, ratio)] for ratio in mask_ratios}
    index = np.arange(len(mask_ratios))
    index_shifted = index + bar_width

    plt.figure(figsize=(10, 6))
    plt.bar(index, [e[0] for e in strategy_errors.values()], width=bar_width, label='Structure Error (MSE)', color='skyblue')
    plt.bar(index_shifted, [e[1] for e in strategy_errors.values()], width=bar_width, label='Degree Error (MSE)', color='orange')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Prediction Error (MSE)')
    plt.title(f'MaskGAE: {strategy.capitalize()} Strategy: Link Prediction Errors for Different Masking Ratios')
    plt.xticks(index + bar_width / 2, [str(r) for r in mask_ratios])
    plt.legend()
    plt.grid(True)
    plt.show()


## Klasyfikacja społeczności

In [ ]:
import torch
import networkx as nx
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from torch_geometric.datasets import Planetoid
from tqdm import tqdm  # Import for progress tracking

CUSTOM_EPOCHS = 45

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')  # Display the device (CPU/GPU)

# Loading the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)  # Move data to the selected device

# Split data into train/test for edges and nodes
def split_edges_and_nodes(data, test_ratio=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)
    num_nodes = data.num_nodes

    # Random permutation of indices
    perm_edges = torch.randperm(num_edges)
    perm_nodes = torch.randperm(num_nodes)
    test_size_edges = int(num_edges * test_ratio)
    test_size_nodes = int(num_nodes * test_ratio)

    # Split indices into train/test
    test_edge_index = edge_index[:, perm_edges[:test_size_edges]]
    train_edge_index = edge_index[:, perm_edges[test_size_edges:]]
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[perm_nodes[test_size_nodes:]] = True
    test_mask[perm_nodes[:test_size_nodes]] = True

    train_data = Data(x=data.x, edge_index=train_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask)
    test_data = Data(x=data.x, edge_index=test_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask)

    return train_data, test_data

train_data, test_data = split_edges_and_nodes(data, test_ratio=0.2)

# Implementing MaskGAE model with classification layer
class MaskGAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_classes):
        super(MaskGAE, self).__init__()

        # GCN Encoder with two layers
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

        # Structure decoder with two MLP layers
        self.structure_decoder = nn.Sequential(
            nn.Linear(out_channels, 2 * out_channels),
            nn.ReLU(),
            nn.Linear(2 * out_channels, in_channels)
        )

        # Degree decoder with two MLP layers
        self.degree_decoder = nn.Sequential(
            nn.Linear(out_channels, 2 * out_channels),
            nn.ReLU(),
            nn.Linear(2 * out_channels, in_channels)
        )

        # Node classification decoder
        self.classifier = nn.Linear(out_channels, num_classes)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode_structure(self, z):
        # Decoding the graph structure
        return self.structure_decoder(z)

    def decode_degree(self, z):
        # Decoding node degrees
        return self.degree_decoder(z)

    def classify(self, z):
        # Node classification
        return self.classifier(z)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        structure_reconstruction = self.decode_structure(z)
        degree_prediction = self.decode_degree(z)
        node_classification = self.classify(z)

        # Return structure reconstruction, degree prediction, and classification output
        return structure_reconstruction, degree_prediction, node_classification

# Masking Strategies

# Masking edges
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()

    data.edge_index = masked_edge_index
    return data

# Masking neighbors
def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()
    data.edge_index = masked_edge_index
    return data

# Masking attributes
def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    data.x[mask_indices] = 0  # Zero out the features of selected nodes
    return data

# Training and evaluating the model with different masking strategies
def train_and_evaluate(in_channels, out_channels, num_classes, train_data, mask_ratios, epochs=100, mask_strategy='mask_edges'):
    results = defaultdict(dict)
    loss_results = {}  # Simple dictionary for losses
    models = {}
    criterion_classification = torch.nn.CrossEntropyLoss()

    for ratio in mask_ratios:
        print(f'Training with mask ratio: {ratio} using {mask_strategy} strategy')

        # Move model to GPU
        model = MaskGAE(in_channels=in_channels, out_channels=out_channels, num_classes=num_classes).to(device)
        optimizer = Adam(model.parameters(), lr=0.01)

        accuracies = []
        losses = []

        # Using tqdm to monitor progress
        for epoch in tqdm(range(epochs), desc=f"Mask Ratio {ratio} ({mask_strategy})"):
            model.train()
            optimizer.zero_grad()

            # Applying the appropriate masking strategy
            if mask_strategy == 'mask_edges':
                masked_train_data = mask_edges(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_neighborhood':
                masked_train_data = mask_neighborhood(train_data.clone(), ratio) if ratio > 0 else train_data
            elif mask_strategy == 'mask_attributes':
                masked_train_data = mask_attributes(train_data.clone(), ratio) if ratio > 0 else train_data

            # Moving the data to the GPU
            masked_train_data = masked_train_data.to(device)

            # Forward pass through the model
            structure_out, degree_out, node_classification = model(masked_train_data.x, masked_train_data.edge_index)
            loss = criterion_classification(node_classification[masked_train_data.train_mask], masked_train_data.y[masked_train_data.train_mask])
            loss.backward()
            optimizer.step()

            losses.append(loss.item())
            tqdm.write(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

            pred = node_classification[masked_train_data.train_mask].argmax(dim=1)
            correct = pred.eq(masked_train_data.y[masked_train_data.train_mask]).sum().item()
            acc = correct / masked_train_data.train_mask.sum().item()
            accuracies.append(acc)

        results[ratio] = accuracies

        if mask_strategy not in loss_results:
            loss_results[mask_strategy] = {}
        loss_results[mask_strategy][ratio] = losses

        models[(mask_strategy, ratio)] = model

    return results, models, loss_results

# Evaluating the classification performance on the test set
def evaluate_classification(model, test_data):
    model.eval()
    with torch.no_grad():
        _, _, node_classification = model(test_data.x, test_data.edge_index)
        pred = node_classification[test_data.test_mask].argmax(dim=1)
        true = test_data.y[test_data.test_mask]
        pred_probs = F.softmax(node_classification[test_data.test_mask], dim=1)[:, 1]

        metrics = calculate_metrics(pred_probs.cpu(), true.cpu())

        print(f'Test Metrics: {metrics}')
        return metrics

# Function to calculate various metrics
def calculate_metrics(pred, true):
    pred_binary = (pred > 0.5).float()
    true_binary = (true > 0.5).float()

    mse = F.mse_loss(pred, true).item()
    f1 = f1_score(true_binary.numpy(), pred_binary.numpy(), average='macro')
    accuracy = accuracy_score(true_binary.numpy(), pred_binary.numpy())
    precision = precision_score(true_binary.numpy(), pred_binary.numpy(), average='macro')
    recall = recall_score(true_binary.numpy(), pred_binary.numpy(), average='macro')

    # Additional metrics for continuous predictions
    roc_auc = roc_auc_score(true_binary.numpy(), pred.numpy(), average='macro')
    avg_precision = average_precision_score(true_binary.numpy(), pred.numpy(), average='macro')

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

# Training and evaluation loop
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
strategies = ['mask_edges', 'mask_neighborhood', 'mask_attributes']
all_results = {}
all_loss_results = {}
all_models = {}

for strategy in strategies:
    results, models, loss_results = train_and_evaluate(
        in_channels=train_data.num_features,
        out_channels=2,
        num_classes=dataset.num_classes,
        train_data=train_data,
        mask_ratios=mask_ratios,
        epochs=CUSTOM_EPOCHS,
        mask_strategy=strategy
    )
    all_results[strategy] = results
    all_loss_results[strategy] = loss_results
    all_models[strategy] = models

# Visualizing classification accuracy results
for strategy in strategies:
    accuracies = {ratio: all_results[strategy][ratio][-1] for ratio in mask_ratios}
    index = np.arange(len(accuracies))
    bar_width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(index, list(accuracies.values()), width=bar_width, color='skyblue', label='Classification Accuracy')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Accuracy')
    plt.title(f'MaskGAE {strategy.capitalize()}: Classification Accuracy for Different Masking Ratios')
    plt.xticks(index, [str(ratio) for ratio in accuracies.keys()])
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot training loss results for each strategy and mask ratio
def plot_loss_results(loss_results):
    print(f"loss_results: {loss_results}")  # Debugowanie, aby zobaczyć strukturę danych
    plt.figure(figsize=(15, 10))
    mask_strategies = loss_results.keys()

    for i, mask_strategy in enumerate(mask_strategies):
        plt.subplot(2, 2, i + 1)

        # Uzyskanie danych dla konkretnej strategii maskowania
        nested_strategy_results = loss_results[mask_strategy]

        # Ponowne zagnieżdżenie - pobranie wewnętrznego słownika
        strategy_results = nested_strategy_results.get(mask_strategy, {})
        if not strategy_results:
            print(f"No data to plot for strategy {mask_strategy}")
            continue

        # Iteracja po współczynnikach maskowania i ich stratach
        for ratio, losses in sorted(strategy_results.items()):
            if isinstance(losses, list) and losses:
                plt.plot(range(len(losses)), losses, label=f'Mask ratio {ratio}')  # Poprawiony zakres dla osi X
            else:
                print(f"No loss data for mask ratio {ratio} in strategy {mask_strategy}")

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'MaskGAE: {mask_strategy.capitalize()} Strategy: Training Loss')
        plt.legend()  # Upewnij się, że legenda jest włączona
        plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_loss_results(all_loss_results)

# Evaluate model performance on the test set
for strategy, models in all_models.items():
    for (mask_strategy, ratio), model in models.items():
        print(f"Evaluating model for strategy '{mask_strategy}' with mask ratio {ratio}")
        evaluate_classification(model, test_data)


# Graf losowy syntetyczny - RGAE

## Przewidywanie połaczeń

In [ ]:
import torch
import networkx as nx
import numpy as np
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
import os
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from torch_geometric.datasets import Planetoid
from tqdm import tqdm

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

CUSTOM_EPOCHS = 45

# Wczytanie zestawu danych Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)

# Podział na zbiór treningowy i testowy
def split_edges(data, test_ratio=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)

    # Wygenerowanie losowego permutacji indeksów
    perm = torch.randperm(num_edges)
    test_size = int(num_edges * test_ratio)

    # Podział indeksów na zbiór testowy i treningowy
    test_edge_index = edge_index[:, perm[:test_size]].to(device)
    train_edge_index = edge_index[:, perm[test_size:]].to(device)

    # Utworzenie dwóch obiektów Data: jeden dla treningu, drugi dla testowania
    train_data = Data(x=data.x.to(device), edge_index=train_edge_index)
    test_data = Data(x=data.x.to(device), edge_index=test_edge_index)

    return train_data, test_data

train_data, test_data = split_edges(data, test_ratio=0.2)

class RobustGraphAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64, dropout=0.5):
        super(RobustGraphAE, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.linear = nn.Linear(hidden_channels, out_channels)
        self.structure_decoder = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, in_channels)
        )
        self.degree_decoder = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, in_channels)
        )
        self.perturbation_layer = nn.Linear(in_channels, in_channels)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.linear(x)

    def decode_structure(self, z):
        return self.structure_decoder(z)

    def decode_degree(self, z):
        return self.degree_decoder(z)

    def generate_perturbation(self, x):
        return torch.tanh(self.perturbation_layer(x))

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        structure_reconstruction = self.decode_structure(z)
        degree_prediction = self.decode_degree(z)
        perturbation = self.generate_perturbation(x)
        perturbed_x = x + perturbation
        perturbed_z = self.encode(perturbed_x, edge_index)
        perturbed_structure_reconstruction = self.decode_structure(perturbed_z)
        perturbed_degree_prediction = self.decode_degree(perturbed_z)
        return structure_reconstruction, degree_prediction, \
               perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction

def robust_graph_ae_loss(model, x, edge_index, alpha=0.5, beta=0.3):
    structure_reconstruction, degree_prediction, \
    perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction = model(x, edge_index)
    structure_loss = F.mse_loss(structure_reconstruction, x)
    degree_loss = F.mse_loss(degree_prediction, x)
    consistency_loss = F.mse_loss(perturbed_structure_reconstruction, structure_reconstruction) + \
                       F.mse_loss(perturbed_degree_prediction, degree_prediction)
    perturbation_regularization = torch.mean(torch.norm(perturbation, dim=-1))
    total_loss = structure_loss + degree_loss + alpha * consistency_loss + beta * perturbation_regularization
    return total_loss

# New masking strategies
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)

    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()

    data.edge_index = masked_edge_index
    return data

def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)

    data.edge_index = masked_edge_index
    return data

def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes

    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)

    mask_indices = torch.tensor(mask_indices, dtype=torch.long).to(device)
    data.x[mask_indices] = 0  # Zero out the features of selected nodes

    return data

# Updated train_and_evaluate function with the new masking strategies
def train_and_evaluate(in_channels, out_channels, train_data, test_data, mask_ratios, epochs=100):
    results = defaultdict(list)
    models = {}
    mask_strategies = {
        'mask_edges': mask_edges,
        'mask_neighborhood': mask_neighborhood,
        'mask_attributes': mask_attributes,
    }

    for strategy_name, strategy_func in mask_strategies.items():
        for ratio in mask_ratios:
            print(f'Training with mask strategy: {strategy_name}, mask ratio: {ratio}')
            model = RobustGraphAE(in_channels=in_channels, out_channels=out_channels).to(device)
            optimizer = Adam(model.parameters(), lr=0.01)

            losses = []

            for epoch in tqdm(range(epochs), desc=f'{strategy_name} - Mask Ratio {ratio}'):
                model.train()
                optimizer.zero_grad()
                masked_train_data = strategy_func(train_data.clone().to(device), ratio)
                loss = robust_graph_ae_loss(model, masked_train_data.x, masked_train_data.edge_index)
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
                tqdm.write(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

            # Store the losses and models after training
            results[(strategy_name, ratio)] = losses
            models[(strategy_name, ratio)] = model

    return results, models

def evaluate_on_test(model, test_data):
    model.eval()
    with torch.no_grad():
        structure_reconstruction, degree_prediction, \
        perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction = model(test_data.x, test_data.edge_index)
        structure_error = F.mse_loss(structure_reconstruction, test_data.x).item()
        degree_error = F.mse_loss(degree_prediction, test_data.x).item()
        print(f'Test Structure Error: {structure_error}, Test Degree Error: {degree_error}')
        return structure_error, degree_error

# Nowa funkcja do obliczania miar
def calculate_metrics(pred, true):
    # Move tensors to CPU and convert to numpy
    true = true.cpu().numpy().flatten()
    pred = pred.cpu().numpy().flatten()

    # Compute ROC curve and optimal threshold
    fpr, tpr, thresholds = roc_curve(true, pred)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # Use optimal threshold to generate binary predictions
    pred_binary = (pred > optimal_threshold).astype(float)
    true_binary = (true > 0.5).astype(float)

    # Calculate various metrics
    mse = F.mse_loss(torch.tensor(pred), torch.tensor(true)).item()
    f1 = f1_score(true_binary, pred_binary, average='macro')
    accuracy = accuracy_score(true_binary, pred_binary)
    precision = precision_score(true_binary, pred_binary, average='macro')
    recall = recall_score(true_binary, pred_binary, average='macro')
    roc_auc = roc_auc_score(true_binary, pred, average='macro')
    avg_precision = average_precision_score(true_binary, pred, average='macro')

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

def plot_results(results):
    # Iterate over each masking strategy
    strategies = set(strategy_name for strategy_name, _ in results.keys())

    for strategy in strategies:
        plt.figure(figsize=(10, 6))

        # Filter results for the current strategy
        for (strategy_name, mask_ratio), losses in results.items():
            if strategy_name == strategy:
                if mask_ratio == 0.0:
                    plt.plot(losses, label=f'Without masking')
                else:
                    plt.plot(losses, label=f'Mask ratio {mask_ratio}')

        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title(f'RGAE: Effect of Masking Strategy {strategy} on MSE Loss')
        plt.legend()
        plt.grid(True)
        plt.show()

def final_loss_comparison(results):
    comparison_results = defaultdict(list)
    for (strategy_name, ratio), losses in results.items():
        comparison_results[strategy_name].append((ratio, losses[-1]))

    for strategy_name, losses in comparison_results.items():
        ratios, final_losses = zip(*sorted(losses))
        plt.figure(figsize=(10, 6))
        plt.bar(ratios, final_losses, color='skyblue')
        plt.xlabel('Mask Ratio')
        plt.ylabel('Final MSE Loss')
        plt.title(f'RGAE: Final MSE Loss for Masking Strategy {strategy_name}')
        plt.grid(True)
        plt.show()

def predict(model, data):
    model.eval()
    with torch.no_grad():
        structure_reconstruction, degree_prediction, \
        perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction = model(data.x, data.edge_index)
        return structure_reconstruction, degree_prediction, perturbation

def calculate_prediction_error(pred_structure, pred_degree, data):
    structure_error = torch.nn.functional.mse_loss(pred_structure, data.x).item()
    degree_error = torch.nn.functional.mse_loss(pred_degree, data.x).item()
    return structure_error, degree_error

# Training and evaluation
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
results, models = train_and_evaluate(
    in_channels=train_data.num_features,
    out_channels=2,
    train_data=train_data,
    test_data=test_data,
    mask_ratios=mask_ratios,
    epochs=CUSTOM_EPOCHS
)

# Plot results
plot_results(results)
final_loss_comparison(results)

# Evaluate on test data
errors = {}
metrics_results = {}
encoded_representations = {}
for (strategy_name, ratio), model in models.items():
    errors[strategy_name] = errors.get(strategy_name, {})
    metrics_results[strategy_name] = metrics_results.get(strategy_name, {})
    encoded_representations[strategy_name] = encoded_representations.get(strategy_name, {})

    test_loss = evaluate_on_test(model, test_data)
    print(f"{strategy_name} | Mask Ratio: {ratio} | Test Loss: {test_loss}")

    pred_structure, pred_degree, perturbation = predict(model, test_data)
    structure_error, degree_error = calculate_prediction_error(pred_structure, pred_degree, test_data)
    errors[strategy_name][ratio] = (structure_error, degree_error)
    encoded_representations[strategy_name][ratio] = pred_structure

    # Calculate additional metrics
    metrics = calculate_metrics(pred_structure, test_data.x)
    metrics_results[strategy_name][ratio] = metrics
    print(f"Metrics for {strategy_name} | Mask Ratio: {ratio}: {metrics}")

# Adjust error plotting to check for key existence
for strategy in strategies:
    strategy_errors = {ratio: errors[strategy].get(ratio, (0, 0)) for ratio in mask_ratios}
    index = np.arange(len(mask_ratios))
    index_shifted = index + bar_width

    plt.figure(figsize=(10, 6))
    plt.bar(index, [e[0] for e in strategy_errors.values()], width=bar_width, label='Structure Error', color='skyblue')
    plt.bar(index_shifted, [e[1] for e in strategy_errors.values()], width=bar_width, label='Degree Error', color='orange')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Prediction Error')
    plt.title(f'RGAE: {strategy.capitalize()} Strategy: Link Prediction Errors for Different Masking Ratios')
    plt.xticks(index + bar_width / 2, [str(r) for r in mask_ratios])
    plt.legend()
    plt.grid(True)
    plt.show()

## Klasyfikacja społeczności

In [ ]:
import torch
import networkx as nx
import numpy as np
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from tqdm import tqdm
from torch_geometric.datasets import Planetoid
import os

# Parameters
CUSTOM_EPOCHS = 100

# Load Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
num_classes = dataset.num_classes
data = dataset[0]

# Ensure CUDA and print the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(f'Using device: {device}')
data = data.to(device)

# Podział na zbiór treningowy i testowy
def split_edges_and_nodes(data, test_ratio=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)
    num_nodes = data.num_nodes

    # Wygenerowanie losowej permutacji indeksów
    perm_edges = torch.randperm(num_edges).to(device)  # Ensure on the same device
    perm_nodes = torch.randperm(num_nodes).to(device)  # Ensure on the same device
    test_size_edges = int(num_edges * test_ratio)
    test_size_nodes = int(num_nodes * test_ratio)

    # Podział indeksów na zbiór testowy i treningowy
    test_edge_index = edge_index[:, perm_edges[:test_size_edges]]
    train_edge_index = edge_index[:, perm_edges[test_size_edges:]]
    train_mask = torch.zeros(num_nodes, dtype=torch.bool, device=device)  # Ensure on the same device
    test_mask = torch.zeros(num_nodes, dtype=torch.bool, device=device)  # Ensure on the same device
    train_mask[perm_nodes[test_size_nodes:]] = True
    test_mask[perm_nodes[:test_size_nodes]] = True

    train_data = Data(x=data.x, edge_index=train_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask).to(device)
    test_data = Data(x=data.x, edge_index=test_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask).to(device)

    return train_data, test_data

train_data, test_data = split_edges_and_nodes(data, test_ratio=0.2)

# Implementacja Autoenkodera Grafowego RobustGraphAE
class RobustGraphAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64, dropout=0.5, num_classes=num_classes):
        super(RobustGraphAE, self).__init__()

        # Enkoder z dwoma warstwami GCN i dodatkową warstwą liniową
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.linear = nn.Linear(hidden_channels, out_channels)

        # Warstwa klasyfikacyjna
        self.classifier = nn.Linear(out_channels, num_classes)

        # Dekoder struktury
        self.structure_decoder = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, in_channels)
        )

        # Dekoder stopnia
        self.degree_decoder = nn.Sequential(
            nn.Linear(out_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, in_channels)
        )

        # Warstwa perturbacji
        self.perturbation_layer = nn.Linear(in_channels, in_channels)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.linear(x)

    def decode_structure(self, z):
        return self.structure_decoder(z)

    def decode_degree(self, z):
        return self.degree_decoder(z)

    def classify(self, z):
        return self.classifier(z)

    def generate_perturbation(self, x):
        return torch.tanh(self.perturbation_layer(x))

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        structure_reconstruction = self.decode_structure(z)
        degree_prediction = self.decode_degree(z)
        class_logits = self.classify(z)

        # Generujemy perturbację wejścia
        perturbation = self.generate_perturbation(x)
        perturbed_x = x + perturbation

        # Przebieg przodu dla danych zakłóconych
        perturbed_z = self.encode(perturbed_x, edge_index)
        perturbed_structure_reconstruction = self.decode_structure(perturbed_z)
        perturbed_degree_prediction = self.decode_degree(perturbed_z)

        return structure_reconstruction, degree_prediction, class_logits, \
               perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction

# Updated robust_graph_ae_loss function
def robust_graph_ae_loss(model, x, edge_index, y, train_mask, alpha=0.5, beta=0.3, gamma=0.2):
    structure_reconstruction, degree_prediction, class_logits, \
    perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction = model(x, edge_index)

    # Strata rekonstrukcji struktury
    structure_loss = F.mse_loss(structure_reconstruction, x)

    # Strata predykcji stopnia
    degree_loss = F.mse_loss(degree_prediction, x)

    # Strata klasyfikacji węzłów
    classification_loss = F.cross_entropy(class_logits[train_mask], y[train_mask])

    # Strata spójności między rekonstrukcjami zakłóconymi i niezakłóconymi
    consistency_loss = F.mse_loss(perturbed_structure_reconstruction, structure_reconstruction) + \
                       F.mse_loss(perturbed_degree_prediction, degree_prediction)

    # Regularizacja perturbacji
    perturbation_regularization = torch.mean(torch.norm(perturbation, dim=-1))

    total_loss = structure_loss + degree_loss + alpha * consistency_loss + beta * perturbation_regularization + gamma * classification_loss

    return total_loss

# New masking strategies
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()

    data.edge_index = masked_edge_index
    return data

def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)
    data.edge_index = masked_edge_index
    return data

def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long).to(device)
    data.x[mask_indices] = 0  # Zero out the features of selected nodes
    return data

# Updated train_and_evaluate function with the new masking strategies
def train_and_evaluate(in_channels, out_channels, train_data, test_data, mask_ratios, epochs=100):
    results = defaultdict(list)
    models = {}
    accuracies = defaultdict(list)
    mask_strategies = {
        'mask_edges': mask_edges,
        'mask_neighborhood': mask_neighborhood,
        'mask_attributes': mask_attributes,
    }

    for strategy_name, strategy_func in mask_strategies.items():
        for ratio in mask_ratios:
            print(f'Training with mask strategy: {strategy_name}, mask ratio: {ratio}')
            model = RobustGraphAE(in_channels=in_channels, out_channels=out_channels).to(device)
            optimizer = Adam(model.parameters(), lr=0.01)

            losses = []

            # Adding tqdm for progress bar
            for epoch in tqdm(range(epochs), desc=f'Training Epochs for {strategy_name} with ratio {ratio}'):
                model.train()
                optimizer.zero_grad()
                masked_train_data = strategy_func(train_data.clone(), ratio).to(device)  # Move to same device
                loss = robust_graph_ae_loss(
                    model,
                    masked_train_data.x,
                    masked_train_data.edge_index,
                    masked_train_data.y,
                    masked_train_data.train_mask,
                )
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

                if epoch % 10 == 0:
                    print(f'Epoch {epoch}, Loss: {loss.item()}')

            # Store the losses and models after training
            results[(strategy_name, ratio)] = losses
            models[(strategy_name, ratio)] = model

            # Evaluate on the test set for classification accuracy
            accuracy = evaluate_classification_accuracy(model, test_data)
            accuracies[strategy_name].append((ratio, accuracy))
            print(f"{strategy_name} | Mask Ratio: {ratio} | Test Accuracy: {accuracy}")

            # Call predict_on_unlabeled to compute metrics on unlabeled data
            predictions, metrics = predict_on_unlabeled(model, test_data)
            print(f"Strategy: {strategy_name}, Mask Ratio: {ratio}, Metrics on Unlabeled Data: {metrics}")

    return results, models, accuracies

# Updated evaluate_classification_accuracy function
def evaluate_classification_accuracy(model, test_data):
    model.eval()
    with torch.no_grad():
        _, _, class_logits, _, _, _ = model(test_data.x, test_data.edge_index)
        pred = class_logits[test_data.test_mask].max(1)[1]
        acc = pred.eq(test_data.y[test_data.test_mask]).sum().item() / test_data.test_mask.sum().item()
        return acc

# Ewaluacja na zbiorze testowym
def evaluate_on_test(model, test_data):
    model.eval()
    with torch.no_grad():
        structure_reconstruction, degree_prediction, class_logits, \
        perturbation, perturbed_structure_reconstruction, perturbed_degree_prediction = model(test_data.x, test_data.edge_index)

        # Structure error
        structure_error = F.mse_loss(structure_reconstruction, test_data.x).item()

        # Degree error
        degree_error = F.mse_loss(degree_prediction, test_data.x).item()

        # Classification accuracy
        pred = class_logits[test_data.test_mask].max(1)[1]
        classification_accuracy = pred.eq(test_data.y[test_data.test_mask]).sum().item() / test_data.test_mask.sum().item()

        print(f'Test Structure Error: {structure_error}, Test Degree Error: {degree_error}, Test Classification Accuracy: {classification_accuracy}')
        return structure_error, degree_error, classification_accuracy

# Nowa funkcja obliczająca miary
def calculate_metrics(pred, true):
    # Użyj argmax do uzyskania przewidywań klas (najwyższa logit dla każdej próbki)
    pred_classes = pred.max(1)[1]

    # MSE obliczane tylko dla wartości logistycznych przewidywań w porównaniu do one-hot encodowanych prawdziwych etykiet
    true_one_hot = F.one_hot(true, num_classes=pred.size(1)).float()
    mse = F.mse_loss(pred, true_one_hot).item()

    # Konwersja tensorów na numpy do użycia w sklearn
    pred_classes_np = pred_classes.cpu().numpy()
    true_np = true.cpu().numpy()

    f1 = f1_score(true_np, pred_classes_np, average='macro')
    accuracy = accuracy_score(true_np, pred_classes_np)
    precision = precision_score(true_np, pred_classes_np, average='macro')
    recall = recall_score(true_np, pred_classes_np, average='macro')

    # Additional metrics for continuous predictions
    if len(np.unique(true_np)) > 1:  # Sprawdzenie, czy więcej niż jedna klasa występuje
        roc_auc = roc_auc_score(true_one_hot.cpu().numpy(), pred.cpu().numpy(), multi_class='ovr')
        avg_precision = average_precision_score(true_one_hot.cpu().numpy(), pred.cpu().numpy())
    else:
        roc_auc = 'undefined'  # Nie można obliczyć AUC, jeśli nie ma więcej niż jedna klasa
        avg_precision = 'undefined'

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

# Predykcja na nieolabelowanych danych
def predict_on_unlabeled(model, data):
    model.eval()
    with torch.no_grad():
        structure_reconstruction, degree_prediction, class_logits, _, _, _ = model(data.x, data.edge_index)
        predictions = class_logits.max(1)[1]

        # Obliczanie metryk
        metrics = calculate_metrics(class_logits, data.y)

        return predictions, metrics

# Run the training and evaluation process
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
results, models, accuracies = train_and_evaluate(
    in_channels=train_data.num_features,
    out_channels=2,
    train_data=train_data,
    test_data=test_data,
    mask_ratios=mask_ratios,
    epochs=CUSTOM_EPOCHS
)

# Analiza wyników i wizualizacje
def plot_results(results, accuracies, mask_ratios):
    for strategy_name in ['mask_edges', 'mask_neighborhood', 'mask_attributes']:
        plt.figure(figsize=(10, 6))
        for (strategy, ratio), losses in results.items():
            if strategy == strategy_name:
                plt.plot(losses, label=f'Mask ratio {ratio}')

        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title(f'RGAE: Training Loss for Masking Strategy: {strategy_name}')
        plt.legend()
        plt.grid(True)
        plt.show()

plot_results(results, accuracies, mask_ratios)

# Wizualizacja wyników klasyfikacji
for strategy in ['mask_edges', 'mask_neighborhood', 'mask_attributes']:
    strategy_accuracies = {ratio: accuracy for ratio, accuracy in accuracies[strategy]}

    index = np.arange(len(strategy_accuracies))
    bar_width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(index, list(strategy_accuracies.values()), width=bar_width, color='skyblue', label='Classification Accuracy')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Accuracy')
    plt.title(f'RGAE: {strategy.capitalize()}: Classification Accuracy for Different Masking Ratios')
    plt.xticks(index, [str(ratio) for ratio in strategy_accuracies.keys()])
    plt.legend()
    plt.grid(True)
    plt.show()

# Graf losowy syntetyczny - S2GAE

## Przewidywanie połaczeń

In [ ]:
import torch
import networkx as nx
import numpy as np
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, roc_curve
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_undirected
from tqdm import tqdm

CUSTOM_EPOCHS = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Wczytanie danych Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
data.edge_index = to_undirected(data.edge_index).to(device)
data = data.to(device)

## Podział na zbiór treningowy i testowy
def split_edges(data, test_ratio=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)

    # Wygenerowanie losowego permutacji indeksów
    perm = torch.randperm(num_edges)
    test_size = int(num_edges * test_ratio)

    # Podział indeksów na zbiór testowy i treningowy
    test_edge_index = edge_index[:, perm[:test_size]]
    train_edge_index = edge_index[:, perm[test_size:]]

    # Utworzenie dwóch obiektów Data: jeden dla treningu, drugi dla testowania
    train_data = Data(x=data.x, edge_index=train_edge_index).to(device)
    test_data = Data(x=data.x, edge_index=test_edge_index).to(device)

    return train_data, test_data

train_data, test_data = split_edges(data, test_ratio=0.2)

## Implementacja Autoenkodera Grafowego S2GAE
class S2GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(S2GAE, self).__init__()

        # Enkoder z dwoma warstwami GCN
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

        # Dekoder z korelacją krzyżową do rekonstrukcji krawędzi
        self.cross_correlation_decoder = nn.Sequential(
            nn.Linear(out_channels * 3, out_channels * 2),
            nn.ReLU(),
            nn.Linear(out_channels * 2, 1)
        )

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, edge_index):
        row, col = edge_index
        h_e = torch.cat([z[row] * z[col], z[row], z[col]], dim=1)
        return self.cross_correlation_decoder(h_e).squeeze()

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        edge_reconstruction = self.decode(z, edge_index)
        return edge_reconstruction

## Nowe funkcje maskowania

# Maskowanie krawędzi
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous()

    data.edge_index = masked_edge_index
    return data

# Maskowanie sąsiedztw
def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)
    data.edge_index = masked_edge_index
    return data

# Maskowanie cech
def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long).to(device)
    data.x[mask_indices] = 0  # Wyzerowanie cech wybranych węzłów
    return data

## Trening modelu z użyciem zbioru treningowego i ocena na zbiorze testowym
def train_and_evaluate(in_channels, out_channels, train_data, test_data, mask_ratios, epochs=100):
    results = defaultdict(lambda: defaultdict(list))
    models = defaultdict(dict)
    criterion = torch.nn.BCEWithLogitsLoss()  # Binary Cross Entropy for link prediction

    # Strategia maskowania
    mask_strategies = {
        'mask_edges': mask_edges,
        'mask_neighborhood': mask_neighborhood,
        'mask_attributes': mask_attributes,
    }

    for strategy_name, strategy_func in mask_strategies.items():
        for ratio in mask_ratios:
            print(f'Training with mask strategy: {strategy_name}, mask ratio: {ratio}')
            # Inicjalizacja nowego modelu dla każdej strategii maskowania i mask_ratio
            model = S2GAE(in_channels=in_channels, out_channels=out_channels).to(device)
            optimizer = Adam(model.parameters(), lr=0.01)

            for epoch in tqdm(range(epochs), desc=f"Training {strategy_name} - ratio {ratio}"):
                model.train()
                optimizer.zero_grad()

                # Maskowanie danych
                masked_train_data = strategy_func(train_data.clone(), ratio) if ratio > 0 else train_data

                edge_reconstruction = model(masked_train_data.x, masked_train_data.edge_index)

                edge_labels = torch.ones(masked_train_data.edge_index.size(1)).to(device)

                loss = criterion(edge_reconstruction, edge_labels)

                loss.backward()
                optimizer.step()

                results[strategy_name][ratio].append(loss.item())
                tqdm.write(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

            models[strategy_name][ratio] = model

    return results, models

# Funkcja do oceny modelu na zbiorze testowym
def evaluate_on_test(model, test_data):
    model.eval()
    with torch.no_grad():
        z = model.encode(test_data.x, test_data.edge_index)
        edge_reconstruction = model.decode(z, test_data.edge_index)
        edge_labels = torch.ones(test_data.edge_index.size(1)).to(device)
        error = torch.nn.functional.binary_cross_entropy_with_logits(edge_reconstruction, edge_labels).item()

        print(f'Test Edge Reconstruction Error: {error}')
        return error

## Analiza wyników i wizualizacje
def plot_results(results):
    for strategy, strategy_losses in results.items():
        plt.figure(figsize=(10, 6))
        for ratio, losses in strategy_losses.items():
            plt.plot(losses, label=f'Mask Ratio {ratio}')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title(f'S2GAE: Effect of Masking Ratios on MSE Loss for {strategy.capitalize()} Strategy')
        plt.legend()
        plt.grid(True)
        plt.show()

def final_loss_comparison(results):
    comparison_results = defaultdict(list)
    for strategy_name, strategy_losses in results.items():
        for ratio, losses in strategy_losses.items():
            comparison_results[strategy_name].append((ratio, losses[-1]))

    for strategy_name, losses in comparison_results.items():
        ratios, final_losses = zip(*sorted(losses))
        plt.figure(figsize=(10, 6))
        plt.bar(ratios, final_losses, color='skyblue')
        plt.xlabel('Mask Ratio')
        plt.ylabel('Final MSE Loss')
        plt.title(f'S2GAE: Final MSE Loss for Masking Strategy {strategy_name.capitalize()}')
        plt.grid(True)
        plt.show()

def predict(model, data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
        pred_structure = model.decode(z, data.edge_index)
        pred_degree = pred_structure
        return pred_structure, pred_degree, z

def calculate_prediction_error(pred_structure, pred_degree, data):
    num_edges = data.edge_index.size(1)

    if pred_structure.size(0) != num_edges:
        raise ValueError("Mismatch in the number of edge predictions.")

    if pred_degree.size(0) != num_edges:
        raise ValueError("Mismatch in the number of edge predictions.")

    edge_labels = torch.ones(num_edges).to(device)
    structure_error = torch.nn.functional.mse_loss(pred_structure, edge_labels).item()
    degree_error = torch.nn.functional.mse_loss(pred_degree, edge_labels).item()

    return structure_error, degree_error

def calculate_metrics(pred, true):
    # Konwersja logitów na prawdopodobieństwa
    pred_prob = torch.sigmoid(pred)
    pred_binary = (pred_prob > 0.5).float()

    # Konwersja tensorów na numpy do obliczania metryk w sklearn
    true_np = true.cpu().numpy()
    pred_np = pred_prob.detach().cpu().numpy()  # Użycie detach() przed konwersją do numpy

    # Obliczanie różnych metryk
    mse = F.mse_loss(pred_prob, true).item()
    f1 = f1_score(true_np, pred_binary.cpu().numpy(), average='macro')
    accuracy = accuracy_score(true_np, pred_binary.cpu().numpy())
    precision = precision_score(true_np, pred_binary.cpu().numpy(), average='macro', zero_division=0)
    recall = recall_score(true_np, pred_binary.cpu().numpy(), average='macro', zero_division=0)

    # Sprawdzamy, czy są co najmniej dwie klasy
    roc_auc = None
    avg_precision = None
    if len(np.unique(true_np)) > 1:  # Przynajmniej dwie klasy muszą istnieć dla ROC AUC
        roc_auc = roc_auc_score(true_np, pred_np)
        avg_precision = average_precision_score(true_np, pred_np)

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

# Trening i ocena modelu
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
results, models = train_and_evaluate(
    in_channels=train_data.num_features,
    out_channels=2,
    train_data=train_data,
    test_data=test_data,
    mask_ratios=mask_ratios,
    epochs=CUSTOM_EPOCHS
)

# Wizualizacja wyników treningu
plot_results(results)
final_loss_comparison(results)

# Ewaluacja na zbiorze testowym dla wszystkich strategii
test_errors = {}
for strategy in models:
    test_errors[strategy] = {}
    for ratio, model in models[strategy].items():
        test_loss = evaluate_on_test(model, test_data)
        test_errors[strategy][ratio] = test_loss
        print(f"Strategy {strategy}, Mask Ratio {ratio} - Test Loss: {test_loss}")

# Predykcja na nieolabelowanych danych dla wszystkich strategii
unlabeled_data = test_data

predictions = {}
errors = {}
encoded_representations = {}
metrics_results = {}

def generate_negative_edges(data, num_neg_edges):
    """Generuje brakujące krawędzie (negatywne) dla przewidywania połączeń."""
    all_possible_edges = set((i, j) for i in range(data.num_nodes) for j in range(i + 1, data.num_nodes))
    existing_edges = set((i.item(), j.item()) for i, j in data.edge_index.t())
    negative_edges = list(all_possible_edges - existing_edges)

    # Losowe wybieranie brakujących krawędzi (negatywnych)
    neg_edge_index = random.sample(negative_edges, num_neg_edges)
    return torch.tensor(neg_edge_index, dtype=torch.long).t().contiguous().to(device)

for strategy in models:
    predictions[strategy] = {}
    errors[strategy] = {}
    encoded_representations[strategy] = {}
    metrics_results[strategy] = {}

    for ratio, model in models[strategy].items():
        print(f'Predicting for strategy {strategy}, mask ratio: {ratio}')

        # Przewidywanie dla istniejących krawędzi
        pred_structure, pred_degree, encoded_rep = predict(model, unlabeled_data)

        structure_error, degree_error = calculate_prediction_error(pred_structure, pred_degree, unlabeled_data)
        errors[strategy][ratio] = (structure_error, degree_error)
        encoded_representations[strategy][ratio] = encoded_rep

        # Generowanie negatywnych krawędzi (brakujących)
        num_neg_edges = unlabeled_data.edge_index.size(1)  # Generowanie tylu brakujących krawędzi, ile istniejących
        neg_edge_index = generate_negative_edges(unlabeled_data, num_neg_edges)

        # Przewidywanie dla brakujących krawędzi
        pred_negative = model.decode(encoded_rep, neg_edge_index)

        # Łączenie wyników dla krawędzi pozytywnych i negatywnych
        preds = torch.cat([pred_structure, pred_negative], dim=0)
        true_labels = torch.cat([torch.ones(pred_structure.size(0)), torch.zeros(pred_negative.size(0))], dim=0).to(device)

        # Obliczanie metryk
        metrics = calculate_metrics(preds, true_labels)
        metrics_results[strategy][ratio] = metrics
        print(f"Metrics for strategy {strategy}, mask ratio {ratio}: {metrics}")


# Wizualizacja błędów predykcji dla wszystkich
bar_width = 0.35

for strategy in models.keys():
    strategy_errors = {ratio: errors[strategy].get(ratio, (0, 0)) for ratio in mask_ratios}
    index = np.arange(len(mask_ratios))
    index_shifted = index + bar_width

    plt.figure(figsize=(10, 6))
    plt.bar(index, [e[0] for e in strategy_errors.values()], width=bar_width, label='Structure Error', color='skyblue')
    plt.bar(index_shifted, [e[1] for e in strategy_errors.values()], width=bar_width, label='Degree Error', color='orange')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Prediction Error')
    plt.title(f'S2GAE: {strategy.capitalize()} Strategy: Link Prediction Errors for Different Masking Ratios')
    plt.xticks(index + bar_width / 2, [str(r) for r in mask_ratios])
    plt.legend()
    plt.grid(True)
    plt.show()


## Klasyfikacja społeczności

In [ ]:
import torch
import networkx as nx
import numpy as np
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch.optim import Adam
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score
from tqdm import tqdm

CUSTOM_EPOCHS = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Załaduj dane Cora z Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')
num_classes = dataset.num_classes
data = dataset[0].to(device)

# Podział na zbiór treningowy i testowy
def split_edges_and_nodes(data, test_ratio=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)
    num_nodes = data.num_nodes

    # Wygenerowanie losowej permutacji indeksów
    perm_edges = torch.randperm(num_edges)
    perm_nodes = torch.randperm(num_nodes)
    test_size_edges = int(num_edges * test_ratio)
    test_size_nodes = int(num_nodes * test_ratio)

    # Podział indeksów na zbiór testowy i treningowy
    test_edge_index = edge_index[:, perm_edges[:test_size_edges]]
    train_edge_index = edge_index[:, perm_edges[test_size_edges:]]
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[perm_nodes[test_size_nodes:]] = True
    test_mask[perm_nodes[:test_size_nodes]] = True

    train_data = Data(x=data.x, edge_index=train_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask)
    test_data = Data(x=data.x, edge_index=test_edge_index, y=data.y, train_mask=train_mask, test_mask=test_mask)

    return train_data, test_data

train_data, test_data = split_edges_and_nodes(data, test_ratio=0.2)

# Implementacja Autoenkodera Grafowego S2GAE z klasyfikatorem węzłów
class S2GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_classes=num_classes):
        super(S2GAE, self).__init__()

        # Enkoder z dwoma warstwami GCN
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

        # Dekoder z korelacją krzyżową do rekonstrukcji krawędzi
        self.cross_correlation_decoder = nn.Sequential(
            nn.Linear(out_channels * 3, out_channels * 2),  # Update input size to match actual concatenated size
            nn.ReLU(),
            nn.Linear(out_channels * 2, 1)  # Output size of 1 to predict edge existence
        )

        # Dekoder do klasyfikacji węzłów
        self.node_classifier = nn.Linear(out_channels, num_classes)  # Assuming num_classes classes for node classification

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, edge_index):
        row, col = edge_index
        # Combine node embeddings from both endpoints of edges
        h_e = torch.cat([z[row] * z[col], z[row], z[col]], dim=1)  # Ensure concatenation results in the expected size
        return self.cross_correlation_decoder(h_e).squeeze()

    def classify(self, z):
        return self.node_classifier(z)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        node_classification = self.classify(z)
        return node_classification

# Nowe funkcje maskowania

# Maskowanie krawędzi
def mask_edges(data, mask_ratio):
    edge_index = data.edge_index.t().tolist()
    num_edges = len(edge_index)
    num_mask = int(num_edges * mask_ratio)

    mask_indices = random.sample(range(num_edges), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long)

    masked_edges = [edge for i, edge in enumerate(edge_index) if i not in mask_indices]
    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)

    data.edge_index = masked_edge_index
    return data

# Maskowanie sąsiedztwa
def mask_neighborhood(data, mask_ratio):
    edge_index = data.edge_index.cpu().numpy()
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    masked_edges = edge_index.T.tolist()

    for _ in range(num_mask):
        node = random.randint(0, num_nodes - 1)
        edges_to_remove = [edge for edge in masked_edges if edge[0] == node or edge[1] == node]
        for edge in edges_to_remove:
            if edge in masked_edges:
                masked_edges.remove(edge)

    masked_edge_index = torch.tensor(masked_edges, dtype=torch.long).t().contiguous().to(device)
    data.edge_index = masked_edge_index
    return data

# Maskowanie cech
def mask_attributes(data, mask_ratio):
    num_nodes = data.num_nodes
    num_mask = int(num_nodes * mask_ratio)
    mask_indices = random.sample(range(num_nodes), num_mask)
    mask_indices = torch.tensor(mask_indices, dtype=torch.long).to(device)
    data.x[mask_indices] = 0  # Zero out the features of selected nodes
    return data

# Funkcja obliczająca różne miary dla predykcji
def calculate_metrics(pred, true):
    pred_binary = (pred > 0.5).float()
    true_binary = (true > 0.5).float()

    mse = F.mse_loss(pred, true).item()
    f1 = f1_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy(), average='macro')
    accuracy = accuracy_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy())
    precision = precision_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy(), average='macro')
    recall = recall_score(true_binary.cpu().numpy(), pred_binary.cpu().numpy(), average='macro')

    # Additional metrics for continuous predictions
    roc_auc = roc_auc_score(true_binary.cpu().numpy(), pred.cpu().numpy(), average='macro')
    avg_precision = average_precision_score(true_binary.cpu().numpy(), pred.cpu().numpy(), average='macro')

    return {
        'MSE': mse,
        'F1 Score': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'ROC AUC': roc_auc,
        'Average Precision': avg_precision
    }

## Trening modelu i ocena klasyfikacji węzłów
def train_and_evaluate_node_classification(in_channels, out_channels, train_data, test_data, mask_ratios, epochs=100):
    results = defaultdict(lambda: defaultdict(list))
    models = defaultdict(dict)
    criterion = torch.nn.CrossEntropyLoss()  # Loss function for node classification

    # Nowe strategie maskowania
    mask_strategies = {
        'mask_edges': mask_edges,
        'mask_neighborhood': mask_neighborhood,
        'mask_attributes': mask_attributes,
    }

    for strategy_name, strategy_func in mask_strategies.items():
        for ratio in mask_ratios:
            print(f'Training with mask strategy: {strategy_name}, mask ratio: {ratio}')

            # Inicjalizacja nowego modelu dla każdej strategii maskowania i mask_ratio
            model = S2GAE(in_channels=in_channels, out_channels=out_channels).to(device)
            optimizer = Adam(model.parameters(), lr=0.01)

            for epoch in tqdm(range(epochs), desc=f'Training {strategy_name} - Ratio {ratio}'):
                model.train()
                optimizer.zero_grad()

                # Maskowanie danych
                masked_train_data = strategy_func(train_data.clone(), ratio).to(device) if ratio > 0 else train_data.to(device)

                z = model.encode(masked_train_data.x.to(device), masked_train_data.edge_index.to(device))  # Przenieś tensory na GPU
                node_classification = model.classify(z)

                loss = criterion(node_classification[masked_train_data.train_mask], masked_train_data.y[masked_train_data.train_mask])

                loss.backward()
                optimizer.step()

                results[strategy_name][ratio].append(loss.item())
                if epoch % 10 == 0:
                    print(f'Epoch {epoch}, Loss: {loss.item()}')

            models[strategy_name][ratio] = model

    return results, models

# Funkcja do oceny klasyfikacji węzłów na zbiorze testowym
def evaluate_node_classification(model, data, strategy, ratio):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
        node_classification = model.classify(z)
        pred = node_classification[data.test_mask].max(1)[1]
        correct = pred.eq(data.y[data.test_mask]).sum().item()
        accuracy = correct / data.test_mask.sum().item()

        print(f'Predicting for strategy {strategy}, mask ratio: {ratio}')

        # Calculate additional metrics
        metrics = calculate_metrics(pred.float(), data.y[data.test_mask].float())
        print(f"Metrics for strategy {strategy}, mask ratio {ratio}: {metrics}")

        return accuracy, metrics

# Wizualizacja wyników treningu
def plot_results(results):
    for strategy, strategy_losses in results.items():
        plt.figure(figsize=(10, 6))
        for ratio, losses in strategy_losses.items():
            plt.plot(losses, label=f'Mask Ratio {ratio}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'S2GAE: Effect of Masking Ratios on Loss for {strategy.capitalize()} Strategy')
        plt.legend()
        plt.grid(True)
        plt.show()

# Trening i ocena modelu
mask_ratios = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]
results, models = train_and_evaluate_node_classification(
    in_channels=train_data.num_features,
    out_channels=2,
    train_data=train_data,
    test_data=test_data,
    mask_ratios=mask_ratios,
    epochs=CUSTOM_EPOCHS
)

# Wizualizacja wyników treningu
plot_results(results)

# Ewaluacja na zbiorze testowym dla wszystkich strategii
classification_accuracies = {}
for strategy in models:
    classification_accuracies[strategy] = {}
    for ratio, model in models[strategy].items():
        accuracy, metrics = evaluate_node_classification(model, test_data, strategy, ratio)
        classification_accuracies[strategy][ratio] = accuracy

# Wizualizacja dokładności klasyfikacji węzłów dla każdej strategii maskowania
for strategy in classification_accuracies.keys():
    accuracies = classification_accuracies[strategy]
    plt.figure(figsize=(10, 6))
    index = np.arange(len(accuracies))
    bar_width = 0.35

    plt.bar(index, list(accuracies.values()), width=bar_width, color='skyblue', label='Classification Accuracy')
    plt.xlabel('Mask Ratio')
    plt.ylabel('Accuracy')
    plt.title(f'S2GAE: Node Classification Accuracy for {strategy.capitalize()} Strategy')
    plt.xticks(index, [str(ratio) for ratio in accuracies.keys()])
    plt.legend()
    plt.grid(True)
    plt.show()